In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as  np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/GiveMeSomeCredit'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **流程**  
1. 背景介绍  
&ensp; 1.1 字段描述  
2. 数据探索  
&ensp; 2.1 数据读取与基本预览  
&ensp; 2.2 查看缺失值  
&ensp; 2.3 查看异常值  
&ensp; 2.4 查看样本类别分布  
&ensp; 2.5 查看数据相关性  
3. 数据预处理  
&ensp; 3.1 处理缺失值  
&ensp; 3.2 处理异常值  
4. 构建模型  
&ensp; 4.1 随机森林  
&ensp; 4.2 XGBoost  
&ensp; 4.3 GBC  
5. 优化调参  
&ensp; 5.1 随机森林调参  
&ensp; 5.2 梯度提升树调参
6. 提交

## **1. 背景介绍**
* * *  
  
Banks play a crucial role in market economies. They decide who can get finance and on what terms and can make or break investment decisions. For markets and society to function, individuals and companies need access to credit. 
Credit scoring algorithms, which **make a guess at the probability of default**, are the method banks use to determine whether or not a loan should be granted. This competition requires participants to improve on the state of the art in credit scoring, by **predicting the probability that somebody will experience financial distress in the next two years.**
The goal of this competition is to build a model that borrowers can use to help make the best financial decisions.
  
    
  
  
  银行在市场经济中发挥着至关重要的作用。 他们决定谁可以获得资金以及以什么条件获得资金，并且可以做出或破坏投资决策。 为了让市场和社会发挥作用，个人和公司需要获得信贷。信用评分算法对违约概率进行猜测，是银行用来确定是否应授予贷款的方法。 这项比赛要求参与者通过预测某人在未来两年内遇到财务困境的可能性来提高信用评分的最新水平。
 



**主要目的：进行数据建模，预测用户未来违约的概率，以此帮助借贷者做出最佳的财务决策。**  

* * *  

####  **1.1 字段描述**

* **SeriousDlqin2yrs：**  
   逾期90天或更严重的人 - 未来两年可能违约 Y/N

* **RevolvingUtilizationOfUnsecuredLines：**  
   信用卡和个人信用额度（不动产和汽车贷款等分期付款债务除外）的总余额除以信用额度之和 - 可用信贷额度比例 percentage

* **age：**  
借款人年龄（年） - 年龄 integer

* **NumberOfTime30-59DaysPastDueNotWorse：**  
借款人逾期30-59天的次数，但在过去两年内没有更糟的情况。 - 逾期30-59天的笔数 integer

* **DebtRatio：**  
月债务、赡养费、生活费除以月总收入 - 负债率 percentage

* **MonthlyIncome：**  
月收入 - 月收入 real

* **NumberOfOpenCreditLinesAndLoans：**  
未偿贷款数量（分期付款，如汽车贷款或抵押贷款）和信贷额度（如信用卡） - 信贷数量 integer

* **NumberOfTimes90DaysLate：**  
借款人逾期90天或以上的次数。 - 逾期90天+的笔数 integer

* **NumberRealEstateLoansOrLines：**  
包括房屋净值信贷额度在内的抵押贷款和房地产贷款数量 - 固定资产贷款数 integer

* **NumberOfTime60-89DaysPastDueNotWorse：**  
借款人逾期60-89天的次数，但在过去两年内没有更糟的情况。 - 逾期60-89天的笔数 integer

* **NumberOfDependents：**  
家庭中不包括自己的受抚养人数量（配偶、子女等） - 家属数量 integer


## **2. 数据探索**
* * * 

#### **2.1 数据读取与基本预览**

In [ ]:
#数据导入
train_df = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv')
test_df = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv')

In [ ]:
#查看训练集前5行数据
train_df.head()   

In [ ]:
#查看测试集前5行数据
test_df.head()    

In [ ]:
#查看训练集信息
train_df.info()   

In [ ]:
#查看测试集信息
test_df.info()    

**观察：**
* 训练集共有15000条数据，测试集总共有101503条数据。  
* 训练集和测试集中MonthlyIncome（月收入）和NumberOfDependents（家庭成员数量）都有缺失值。

#### **2.2 查看缺失值**

In [ ]:
 #具体查看训练集缺失值
train_df.isnull().sum()  

In [ ]:
#具体查看测试集缺失值
test_df.isnull().sum()    

**观察**
* 训练集和测试集的缺失值数量都占比较大，所以不能直接删除。  
* **MonthlyIncome**属于连续型数值特征，且缺失较多，一般采用平均数填补缺失值。  
* **NumberOfDependents**可以用**众数或中位数**填充空值。

#### **2.3 查看异常值**

In [ ]:
#描述性统计
train_df.describe([0.01,0.1,0.25,.5,.75,.9,.99])

In [ ]:
#再查看一下箱型图
x1=train_df['RevolvingUtilizationOfUnsecuredLines']
x2=train_df['DebtRatio']
x3=train_df['age']
x4=train_df['NumberOfTime30-59DaysPastDueNotWorse']
x5=train_df['NumberOfTime60-89DaysPastDueNotWorse']
x6=train_df['NumberOfTimes90DaysLate']
x7=train_df['NumberOfOpenCreditLinesAndLoans']
x8=train_df['NumberRealEstateLoansOrLines']

fig=plt.figure(figsize=(20,15))
ax1=fig.add_subplot(221)
ax2=fig.add_subplot(222)
ax3=fig.add_subplot(223)
ax4=fig.add_subplot(224)

ax1.boxplot([x1,x2])
ax1.set_xticklabels(["RevolvingUtilizationOfUnsecuredLines","DebtRatio"], fontsize=20)

ax2.boxplot(x3)
ax2.set_xticklabels(["age"], fontsize=20)

ax3.boxplot([x4,x5,x6])
ax3.set_xticklabels(["30-59Days","60-89Days","90Days"], fontsize=20)

ax4.boxplot([x7,x8])
ax4.set_xticklabels(["CreditLinesAndLoans","RealEstateLoans"], fontsize=20)

**观察：**
* 训练集和测试集RevolvingUtilizationOfUnsecuredLines在75%值为0.55，最大值却为50578，可能分布不均或是异常值存在。  
* DebtRatio也是同样的问题。  
* NumberOf-Time30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate, NumberOfTime60-89DaysPastDueNotWorse三种的最大值都是98。  
* 训练集age最小值存在0，有异常值。  
* 家属数量最大值有43。

In [ ]:
#具体查看一下age的情况
train_df.loc[train_df['age'] < 18]   

**观察：**
* 有一条年龄为0的数据，显然是异常值，可以用中位数填充处理

In [ ]:
# 通过箱线图，可以看到三个逾期天数指标（逾期30-59天、逾期60-80天，逾期90天）存在较严重的离群值，且这三个指标的99%分位数与max相差过大，存在异常。
# 具体查看一下这三个逾期指标超过90天的数据。

train_df[train_df['NumberOfTime30-59DaysPastDueNotWorse']>90].shape
train_df[train_df['NumberOfTime60-89DaysPastDueNotWorse']>90].shape
train_df[train_df['NumberOfTimes90DaysLate']>90].shape

**观察：**
* 维度均为（269，12），可以认为离群值均发生在相同行，可以将异常点删去

#### **2.4 查看样本类别分布**

In [ ]:
# 画图查看类别分布
plt.figure()
sns.countplot('SeriousDlqin2yrs',data=train_df)

**观察：**
* 样本标签分类严重不平衡，可能会造成预测性能下降。

#### **2.5 查看数据相关性**

In [ ]:
#画出数据关系热力图
correlation = train_df.corr()
f , ax = plt.subplots(figsize = (13, 13))
plt.title('heatmap',y=1,size=16)
sns.heatmap(correlation,annot = True,  vmax=0.8)

**观察：**
* 由上图可见，NumberOfTime30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate, NumberOfTime60-89DaysPastDueNotWorse三者相关性很大

In [ ]:
 #观察图，年龄和收入分布皆近似正态分布
age=train_df['age']
sns.distplot(age)
plt.show()

mi=train_df[train_df['MonthlyIncome']<50000]['MonthlyIncome']
sns.distplot(mi)
plt.show() 

**观察：**
* 年龄和收入分布近似正态分布。

## **3. 数据预处理** 

#### **3.1 处理缺失值**

In [ ]:
#去除重复数据
train_df.drop_duplicates(inplace=True)    

In [ ]:
#对未命名的第一列重命名为ID，方便观察。
train_df.rename(columns={'Unnamed: 0':'ID'}, inplace=True)
test_df.rename(columns={'Unnamed: 0':'ID'}, inplace=True)

In [ ]:
#只有一个年龄小于18，用中位数替代
train_df.loc[train_df['age'] == 0, 'age'] = train_df['age'].median()

In [ ]:
#查看NumberOfDependents（家庭成员数量）非空值统计
train_df['NumberOfDependents'].value_counts()

In [ ]:
#用中位数填充NumberOfDependents（家庭成员数量）空值
train_df['NumberOfDependents'].fillna(train_df['NumberOfDependents'].median(), inplace=True)
test_df['NumberOfDependents'].fillna(train_df['NumberOfDependents'].median(), inplace=True)

In [ ]:
#家属人数大于8的设为8
k = 0
for i in train_df['NumberOfDependents']:
    if i>8:
        train_df['NumberOfDependents'].values[k]=8
    k +=1
k = 0
for i in test_df['NumberOfDependents']:
    if i>8:
        test_df['NumberOfDependents'].values[k]=8
    k +=1

In [ ]:
# 1.用平均数填充MonthlyIncome(月收入)空值
train_df['MonthlyIncome'] = train_df['MonthlyIncome'].replace(np.nan,train_df['MonthlyIncome'].mean())
test_df['MonthlyIncome'] = test_df['MonthlyIncome'].replace(np.nan,test_df['MonthlyIncome'].mean())

In [ ]:
# # 2.用随机森林填充MonthlyIncome(月收入)空值
# from sklearn.ensemble import RandomForestRegressor
# def set_missing_income(df):
# #把数值型特征都放到随机森林里面去
#     Income_df=df.iloc[:,[6,1,2,3,4,5,7,8,9]]
#     known= Income_df[Income_df.MonthlyIncome.notnull()].values
#     unknown= Income_df[Income_df.MonthlyIncome.isnull()].values
#     y=known[:,0]#y是月收入，第一列数据
#     x=known[:,1:]#x是特征属性值，后面几列
#     rfr=RandomForestRegressor(random_state=0,n_estimators=200,n_jobs=-1)
#     #根据已有数据去拟合随机森林模型
#     rfr.fit(x,y)
#     #预测缺失值
#     predictedIncome =rfr.predict(unknown[:, 1:]).round(0)
#     #填补缺失值
#     df.loc[(df.MonthlyIncome.isnull()),'MonthlyIncome'] = predictedIncome
    
#     return df
# train_df=set_missing_income(train_df)

# def set_missing_income1(df):
# #把数值型特征都放到随机森林里面去
#     Income_df=df.iloc[:,[6,2,3,4,5,7,8,9]]
#     known= Income_df[Income_df.MonthlyIncome.notnull()].values
#     unknown= Income_df[Income_df.MonthlyIncome.isnull()].values
#     y=known[:,0]#y是月收入，第一列数据
#     x=known[:,1:]#x是特征属性值，后面几列
#     rfr=RandomForestRegressor(random_state=0,n_estimators=200,n_jobs=-1)
#     #根据已有数据去拟合随机森林模型
#     rfr.fit(x,y)
#     #预测缺失值
#     predictedIncome =rfr.predict(unknown[:, 1:]).round(0)
#     #填补缺失值
#     df.loc[(df.MonthlyIncome.isnull()),'MonthlyIncome'] = predictedIncome
    
#     return df
# test_df=set_missing_income1(test_df)

# Private Score：0.86641 Public Score：0.85969  
# 测试集效果降低
# 最终采用平均数填充

#### **3.2 处理异常值**

In [ ]:
# 删除NumberOf-Time30-59DaysPastDueNotWorse(逾期30-59), NumberOfTimes90Days-Late（逾期60-89）, NumberOfTime60-89DaysPastDueNotWorse（逾期90）大于90的点
train_df = train_df[train_df['NumberOfTime30-59DaysPastDueNotWorse'] < 90] 
train_df = train_df[train_df['NumberOfTimes90DaysLate'] < 90] 
train_df = train_df[train_df['NumberOfTime60-89DaysPastDueNotWorse'] < 90] 

## **4. 构建模型**
* * * 

In [ ]:
#划分训练集和测试集。
from sklearn.model_selection import train_test_split
x = train_df.drop(['SeriousDlqin2yrs', 'ID'],axis=1)
y= train_df['SeriousDlqin2yrs']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
#auc绘图
def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1],[0,1], "k--") # 画直线做参考
    plt.axis([0,1,0,1])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive rate")

#### **4.1 随机森林**

In [ ]:
# #0.8630
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from  sklearn.ensemble import RandomForestClassifier
#rfc = RandomForestClassifier(n_estimators=500, max_depth=6, class_weight='balanced',random_state=42)
rfc = RandomForestClassifier(n_estimators=500, max_depth=8, class_weight='balanced',random_state=42)
rfc.fit(x_train,y_train)
pred=rfc.predict_proba(x_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr,tpr)
plot_roc_curve(fpr,tpr)
print ('AUC Score :', roc_auc)

#### **4.2 XGBoost**

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth=5,eval_metric='auc',objective='binary:logistic')
model.fit(x_train, y_train)
# make predictions for test data
y_pred = model.predict_proba(x_test)[:,1]
# evaluate predictions
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr,tpr)
print ('AUC Score :', roc_auc)
plot_roc_curve(fpr,tpr)

#### **4.3 梯度提升树**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc_clf_submission = GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, max_depth=8)
gbc_clf_submission.fit(x_train,y_train)
gbc_clf_proba = gbc_clf_submission.predict_proba(x_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, gbc_clf_proba)
roc_auc = auc(fpr,tpr)
print ('AUC Score :', roc_auc)
plot_roc_curve(fpr,tpr)

## **5.优化调参**
* * * 

#### **5.1 随机森林调参**

In [ ]:
#对随机森林模型进行网格调参，采用10折交叉验证
# from sklearn.model_selection import GridSearchCV, StratifiedKFold
# RFC = RandomForestClassifier(n_jobs=-1)
# rf_param_grid = {
# 'max_depth' : [5,6,7,8],
# "min_samples_split": [2, 3, 5],
# "min_samples_leaf": [2, 3, 5],
# "n_estimators" :[300, 500, 700],
# "criterion": ["gini"]
# }
# # Cross validate model with Kfold stratified cross val
# kfold = StratifiedKFold(n_splits=10)
# gsRFC = GridSearchCV(RFC, param_grid=rf_param_grid, cv=kfold, scoring="accuracy", verbose=0)
# gsRFC.fit(x_train, y_train)
# RFC_best = gsRFC.best_estimator_
# # Best score
# print("10折CV准确率：{}".format(round(gsRFC.best_score_, 2))) 
# print("训练集上的准确率：{}".format(round(RFC_best.score(x_train, y_train), 2)))
#0.94
#0.94

* 参考Titanic超参数调节对随机森林模型进行网格调参，运行时长>5h,最终在测试集效果不如预期。  
* **private：0.86439    &ensp; public：0.85856**  
* 模型在训练集和测试集分数相异较大，可能是过拟合。  
* 接下来对GBC模型进行调参

#### **5.2 梯度提升树调参**

In [ ]:
# #对迭代次数网格化调参。
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# param_test1 = {'n_estimators':range(20,81,10)}
# gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
#                                   min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10),
#                        param_grid = param_test1, scoring='roc_auc',cv=5)
# gsearch1.fit(x_train,y_train)
# means = gsearch1.cv_results_['mean_test_score']
# params = gsearch1.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))

* 先确定一个迭代次数
* 对max_depth和min_samples_split进行网格搜索。

In [ ]:
# param_test2 = {'max_depth':range(3,10,2), 'min_samples_split':range(100,801,200)}
# gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50, min_samples_leaf=20, 
#       max_features='sqrt', subsample=0.8, random_state=10), 
#    param_grid = param_test2, scoring='roc_auc', cv=5)
# gsearch2.fit(x_train,y_train)
# means = gsearch2.cv_results_['mean_test_score']
# params = gsearch2.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))

* 暂时定下max_depth=7  
* 再对min_samples_split和min_samples_leaf一起调参

In [ ]:
# param_test3 = {'min_samples_split':range(800,1900,200), 'min_samples_leaf':range(60,101,10)}
# gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50,max_depth=7,
#                                      max_features='sqrt', subsample=0.8, random_state=10), 
#                        param_grid = param_test3, scoring='roc_auc', cv=5)
# gsearch3.fit(x_train,y_train)
# means = gsearch3.cv_results_['mean_test_score']
# params = gsearch3.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))

* 对max_features网格搜索

In [ ]:
# param_test4 = {'max_features':range(1,9)}
# gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50,max_depth=7, min_samples_leaf =120, 
#                min_samples_split =1000, subsample=0.8, random_state=10), 
#                        param_grid = param_test4, scoring='roc_auc', cv=5)
# gsearch4.fit(x_train,y_train)
# means = gsearch4.cv_results_['mean_test_score']
# params = gsearch4.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))

* 对sub_sample网格搜索

In [ ]:
# param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
# gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50,max_depth=7, min_samples_leaf =120, 
#                min_samples_split =1000, max_features=4, random_state=10), 
#                        param_grid = param_test5, scoring='roc_auc', cv=5)
# gsearch5.fit(x_train,y_train)
# means = gsearch5.cv_results_['mean_test_score']
# params = gsearch5.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))

* 确定最终模型

In [ ]:
# 0.8683
gbm1 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1000,max_depth=8, min_samples_leaf =100, 
               min_samples_split =1200, max_features='sqrt', subsample=0.8, random_state=10)
gbm1.fit(x_train,y_train)
gbc_clf_proba = gbm1.predict_proba(x_test)[:,1]
y_pred = gbm1.predict(x_test)
fpr, tpr, _ = roc_curve(y_test, gbc_clf_proba)
roc_auc = auc(fpr,tpr)
plot_roc_curve(fpr,tpr)
print ('AUC Score :', roc_auc)

In [ ]:
#绘制图像查看特征关注度
train_df=train_df.drop(['SeriousDlqin2yrs', 'ID'],axis=1)
def plot_feature_importances(model):
    plt.figure(figsize=(10,8))
    n_features = train_df.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), train_df.columns)
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')
    plt.ylim(-1, n_features)

In [ ]:
#查看不同模型对各个特征的关注度
plot_feature_importances(rfc)
plot_feature_importances(model)
plot_feature_importances(gbm1)

## **6. 提交**
* * * 

In [ ]:
#最后使用调参过后的GBC模型提交
test_df = test_df.drop(['SeriousDlqin2yrs', 'ID'],axis=1)
gbc_clf_proba = gbm1.predict_proba(test_df)[:,1]
# rfc_proba = RFC_best.predict_proba(test_df)[:,1]
ids = np.arange(1,101504)
submission = pd.DataFrame( {'Id': ids, 'Probability':gbc_clf_proba})
submission.to_csv("submision.csv", index=False)